In [ ]:
!pip install transformers
!pip install datasets
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.layers import Concatenate, Dense, Input
from tensorflow.keras.models import Model
from transformers import BertTokenizer, TFBertModel

In [ ]:
def read_text_file(file_path):
    text_list = []

    with open(file_path, 'r') as file:
        text = ''.join(file.readlines()) #read and concate all line

    text_list.append(text)
    return text_list

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Qna Project/ch16ipc.txt'
ipc_text = read_text_file(file_path)


print(ipc_text[0][1500:2000])

sh. B fires and kills Z. Here B may be guilty of no offence; but A has committed the offence of culpable homicide.
(c) A, by shooting at a fowl with intent to kill and steal it, kills B, who is behind a bush; A not knowing that he was there. Here, although A was doing an unlawful act, he was not guilty of culpable homicide, as he did not intend to kill B, or to cause death by doing an act that he knew was likely to cause death.
Explanation 1 - A person who causes bodily injury to another who is 


In [ ]:
def segment_documents(docs, max_doc_length=450):
  segmented_docs = []

  for doc in docs:
    split_to_words = doc.split(" ")

    if len(split_to_words) > max_doc_length:
      for doc_segment in range(0, len(split_to_words), max_doc_length):
        segmented_docs.append( " ".join(split_to_words[doc_segment:doc_segment + max_doc_length])) #split it into smaller segments

    else: #shorter len of doc
      segmented_docs.append(doc)

  return segmented_docs

In [ ]:
segment_documents(ipc_text,200)[0],segment_documents(ipc_text,200)[1]

('CHAPTER XVI\nOF OFFENCESAFFECTINGTHE HUMAN BODY\nOf offences affecting life\nSection 299 : Culpable homicide\nSection 300 : Murder\nSection 301 : Culpable homicide by causing death of person other than person whose death was intended\nSection 302 : Punishment for murder\nSection 303 : Punishment for murder by life-convict\nSection 304 : Punishment for culpable homicide not amounting to murder\nSection 304A : Causing death by negligence\nSection 304B : Dowry death\nSection 305 : Abetment of suicide of child or insane person\nSection 306 : Abetment of suicide\nSection 307 : Attempt to murder\nSection 308 : Attempt to commit culpable homicide\nSection 309 : Attempt to commit suicide\nSection 310 : Thug\nSection 311 : Punishment\n\n/n\n\nSection 299 of The Indian Penal Code, 1860: Culpable homicide\nBare Act\nWhoever causes death by doing an act with the intention of causing death, or with the intention of causing such bodily injury as is likely to cause death, or with the knowledge that

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Embeddings

***'with the knowledge that he is likely'***


---


### into token:

'with' 'the' 'knowledge' 'that' 'he' 'is' 'likely'


---


### substitute token with ids:
 array([**[ 101, 2007, 1996, 3716, 2008, 2002, 2003, 3497,  102]**],


---


### into model and then it generate
[bert embedding]

3d tensor

output would be a vector for each input token and each vector is made up of 768 numbers (floats).

In [ ]:
text='with the knowledge that he is likely'
encoded_text = tokenizer(text, padding=True, truncation=True, return_tensors="tf")
encoded_text

{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=
array([[ 101, 2007, 1996, 3716, 2008, 2002, 2003, 3497,  102]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
output = bert_model(encoded_text)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 9, 768), dtype=float32, numpy=
array([[[-6.4337581e-02,  5.6585990e-02,  6.2800542e-02, ...,
         -2.0759617e-01,  6.3057467e-02,  1.5633874e-01],
        [-2.1881455e-01,  6.4207786e-01,  2.4918632e-01, ...,
         -1.4061040e-01, -2.5251187e-02,  7.0201331e-01],
        [-3.4663269e-01, -3.7135262e-02,  4.9111846e-01, ...,
         -1.6414827e-01,  8.5437149e-03, -6.5428123e-02],
        ...,
        [-2.0576940e-01, -4.9344361e-01,  2.4325705e-01, ...,
          6.6516750e-02,  3.5572445e-01,  1.4333014e-01],
        [ 1.5940818e-01, -1.1328080e+00, -2.7910131e-01, ...,
          1.0658249e-01, -9.0237893e-03, -6.6733760e-01],
        [ 7.9656202e-01,  1.8897083e-01, -3.7848440e-01, ...,
          4.1012093e-04, -6.3792038e-01, -3.2317972e-01]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-0.8018575 , -0.225901  ,  0.18871751,  0.6019103 , -0.210

In [ ]:
pooled_output = output.pooler_output
pooled_output.numpy()

array([[-0.80185735, -0.22590084,  0.18871793,  0.60191005, -0.21091665,
        -0.07215006,  0.8358946 ,  0.18288662,  0.00485797, -0.999652  ,
         0.12644693,  0.28555322,  0.9572917 ,  0.06168538,  0.88012296,
        -0.5228098 , -0.06033653, -0.5303301 ,  0.27228633, -0.6663901 ,
         0.55562496,  0.9337207 ,  0.5187825 ,  0.2483487 ,  0.30024287,
         0.404423  , -0.45897213,  0.8479909 ,  0.9168047 ,  0.59454066,
        -0.5839967 ,  0.132411  , -0.95866424, -0.18029517,  0.11202769,
        -0.9673738 ,  0.12568617, -0.6638023 ,  0.01493043,  0.01872515,
        -0.7676907 ,  0.16341369,  0.9940873 , -0.3506261 , -0.09061731,
        -0.2827453 , -0.99964374,  0.19422267, -0.78106314, -0.13232003,
         0.09160349, -0.4589338 ,  0.12383313,  0.31670672,  0.32545543,
         0.38372916, -0.15040788,  0.14456718, -0.07142986, -0.43704674,
        -0.5799241 ,  0.32731643,  0.03345872, -0.8341754 , -0.19264112,
        -0.2524952 , -0.10671038, -0.21316573, -0.0

In [ ]:
def get_embeddings(texts):
    encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors="tf") #tokenizing and then encoding txt
    outputs = bert_model(encoded_texts)
    pooled_output = outputs.pooler_output #output embeddings

    return pooled_output.numpy()

Retrieval

In [ ]:
query = "What is the punishment for murder in Indian Penal Code?"
docs = [
    "The punishment for murder under section 302 of the Indian Penal Code",
    "IPC section 302 deals with the punishment for murder.",
    "Murder is penalized under section 302 of the Indian Penal Code."
]
vectorizer = TfidfVectorizer(analyzer="word", stop_words='english')

tfidf_query = vectorizer.fit_transform([query])
tfidf_docs = vectorizer.transform(docs)
tfidf_query, tfidf_docs

(<1x5 sparse matrix of type '<class 'numpy.float64'>'
 	with 5 stored elements in Compressed Sparse Row format>,
 <3x5 sparse matrix of type '<class 'numpy.float64'>'
 	with 11 stored elements in Compressed Sparse Row format>)

In [ ]:
bert_query_embedding = get_embeddings([query])
bert_doc_embeddings = get_embeddings(docs)
bert_query_embedding,bert_doc_embeddings

(array([[-0.9099835 , -0.54075897, -0.8783243 ,  0.8026237 ,  0.55223995,
         -0.3067179 ,  0.86624193,  0.46545127, -0.7520621 , -0.9999878 ,
         -0.34237322,  0.928218  ,  0.9842718 ,  0.65412706,  0.88884604,
         -0.7881376 , -0.46010745, -0.68923223,  0.39614868, -0.2555447 ,
          0.70666516,  0.9999747 , -0.03110149,  0.44795057,  0.5560621 ,
          0.95811945, -0.79561496,  0.90917563,  0.96409076,  0.71670455,
         -0.7326286 ,  0.40328497, -0.9900527 , -0.30546036, -0.89868927,
         -0.9942048 ,  0.5222275 , -0.6947935 , -0.09650877, -0.14568755,
         -0.85699457,  0.4005583 ,  0.99996793, -0.3067792 ,  0.5562661 ,
         -0.4457666 , -1.        ,  0.44470546, -0.84988356,  0.8078253 ,
          0.8378973 ,  0.82094336,  0.3177794 ,  0.5751103 ,  0.6244037 ,
         -0.47446695,  0.03795572,  0.294045  , -0.4852089 , -0.64029676,
         -0.7261535 ,  0.61106515, -0.7885071 , -0.89965147,  0.80789304,
          0.76954496, -0.36780304, -0.

In [ ]:
combined_query = np.concatenate((tfidf_query.toarray(), bert_query_embedding), axis=1)
combined_docs = np.concatenate((tfidf_docs.toarray(), bert_doc_embeddings), axis=1)
combined_query,combined_docs

(array([[ 0.4472136 ,  0.4472136 ,  0.4472136 ,  0.4472136 ,  0.4472136 ,
         -0.90998352, -0.54075897, -0.87832427,  0.80262369,  0.55223995,
         -0.3067179 ,  0.86624193,  0.46545127, -0.75206208, -0.99998778,
         -0.34237322,  0.92821801,  0.98427182,  0.65412706,  0.88884604,
         -0.78813761, -0.46010745, -0.68923223,  0.39614868, -0.25554469,
          0.70666516,  0.99997473, -0.03110149,  0.44795057,  0.5560621 ,
          0.95811945, -0.79561496,  0.90917563,  0.96409076,  0.71670455,
         -0.73262858,  0.40328497, -0.9900527 , -0.30546036, -0.89868927,
         -0.99420482,  0.52222753, -0.69479352, -0.09650877, -0.14568755,
         -0.85699457,  0.40055829,  0.99996793, -0.30677921,  0.55626613,
         -0.4457666 , -1.        ,  0.44470546, -0.84988356,  0.80782533,
          0.8378973 ,  0.82094336,  0.31777939,  0.57511032,  0.62440372,
         -0.47446695,  0.03795572,  0.294045  , -0.4852089 , -0.64029676,
         -0.72615349,  0.61106515, -0.

In [ ]:
score = cosine_similarity(combined_query, combined_docs)
score

array([[0.93661586, 0.96697227, 0.97943   ]])

In [ ]:
sorted_list = sorted(enumerate(score[0]), key=lambda x: x[1], reverse=True)
sorted_list

[(2, 0.9794299996707087), (1, 0.9669722689630518), (0, 0.9366158632413213)]

In [ ]:
k=2
top_doc_indices = [x[0] for x in sorted_list[:k]]
top_doc_indices

[2, 1]

In [ ]:
top_docs = [docs[x] for x in top_doc_indices]
top_docs

['Murder is penalized under section 302 of the Indian Penal Code.',
 'IPC section 302 deals with the punishment for murder.']

In [ ]:
def combine_tfidf_with_embeddings(query, docs):
    vectorizer = TfidfVectorizer(analyzer="word", stop_words='english')

    tfidf_query = vectorizer.fit_transform([query]) #TF-IDF vectors
    tfidf_docs = vectorizer.transform(docs)

    bert_query_embedding = get_embeddings([query])
    bert_doc_embeddings = get_embeddings(docs)

    combined_query = np.concatenate((tfidf_query.toarray(), bert_query_embedding), axis=1)
    combined_docs = np.concatenate((tfidf_docs.toarray(), bert_doc_embeddings), axis=1)

    return combined_query, combined_docs

In [ ]:
def get_top_k_articles(query, docs, k):
    combined_query, combined_docs = combine_tfidf_with_embeddings(query, docs)
    scores = cosine_similarity(combined_query, combined_docs)

    sorted_list = sorted(enumerate(scores[0]), key=lambda x: x[1], reverse=True)
    top_doc_indices = [x[0] for x in sorted_list[:k]]
    top_docs = [docs[x] for x in top_doc_indices]
    return top_docs

In [ ]:
query = "What is point of discussion of section 302 of the indian penal code?"
# query = "Which section of IPC caters for the punishment of murder?"
k=4
segmented_docs = segment_documents(ipc_text,200)
top_docs = get_top_k_articles(query, segmented_docs,k)
print("Top documents:")
for i,doc in enumerate(top_docs):
    print("----------------------\n",'Doc-',i+1,"\n----------------------\n", doc)


Top documents:
----------------------
 Doc- 1 
----------------------
 act knows that it is so imminently dangerous that it must, in all probability, cause death, or such bodily injury as is likely to cause death, and commits such act without any excuse for incurring the risk of causing death or such injury as aforesaid.

Illustrations

(a) A shoots Z with the intention of killing him. Z dies in consequence. A commits murder.
(b) A, knowing that Z is labouring under such a disease that a blow is likely to cause his death, strikes him with the intention of causing bodily injury. Z dies in consequence of the blow. A is guilty of murder, although the blow might not have been sufficient in the ordinary course of nature to cause the death of a person in a sound state of health. But if A, not knowing that Z is labouring under any disease, gives him such a blow as would not in the ordinary course of nature kill a person in a sound state of health, here A, although he may intend to cause bodil